### Imports

In [2]:
import os

# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Verify if it's set correctly
print(os.environ['CUDA_VISIBLE_DEVICES'])

0


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import pandas as pd
import csv
import sys
import io
import re
import logging
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
!huggingface-cli login

In [7]:
from huggingface_hub import login

In [8]:
login()

In [4]:
model= "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

### Defining questionnarie data and scales

In [5]:
likert_scale = [
    "1 - Strongly Disagree",
    "2 - Disagree",
    "3 - Neutral",
    "4 - Agree",
    "5 - Strongly Agree"
]
likert_scale_reversed = [
   "5 - Strongly Agree",
   "4 - Agree",
   "3 - Neutral",
   "2 - Disagree",
   "1 - Strongly Disagree"
]
Ques44 = ["Is talkative",
    "Tends to find fault with others",
    "Does a thorough job",
    "Is depressed, blue",
    "Is original, comes up with new ideas",
    "Is reserved",
    "Is helpful and unselfish with others",
    "Can be somewhat careless",
    "Is relaxed, handles stress well",
    "Is curious about many different things",
    "Is full of energy",
    "Starts quarrels with others",
    "Is a reliable worker",
    "Can be tense",
    "Is ingenious, a deep thinker",
    "Generates a lot of enthusiasm",
    "Has a forgiving nature",
    "Tends to be disorganized",
    "Worries a lot",
    "Has an active imagination",
    "Tends to be quiet",
    "Is generally trusting",
    "Tends to be lazy",
    "Is emotionally stable, not easily upset",
    "Is inventive",
    "Has an assertive personality",
    "Can be cold and aloof",
    "Perseveres until the task is finished",
    "Can be moody",
    "Values artistic, aesthetic experiences",
    "Is sometimes shy, inhibited",
    "Is considerate and kind to almost everyone",
    "Does things efficiently",
    "Remains calm in tense situations",
    "Prefers work that is routine",
    "Is outgoing, sociable",
    "Is sometimes rude to others",
    "Makes plans and follows through with them",
    "Gets nervous easily",
    "Likes to reflect, play with ideas",
    "Has few artistic interests",
    "Likes to cooperate with others",
    "Is easily distracted",
    "Is sophisticated in art, music, or Literature"]
valid_set = set(['1', '2', '3', '4', '5'])
Ques10= [
    "is reserved",
    "is generally trusting",
    "tends to be lazy",
    "is relaxed, handles stress well",
    "has few artistic interests",
    "is outgoing, sociable",
    "tends to find fault with others",
    "does a thorough job",
    "gets nervous easily",
    "has an active imagination"
]
Ques27 = [
    "It's not wise to tell your secrets",
    "I like to use clever manipulation to get my way",
    "Whatever it takes, you must get the important people on your side",
    "Avoid direct conflict with others because they may be useful in the future",
    "It's wise to keep track of information that you can use against people later",
    "You should wait for the right time to get back at people",
    "There are things you should hide from other people to preserve your reputation",
    "Make sure your plans benefit yourself, not others",
    "Most people can be manipulated",
    "People see me as a natural leader",
    "I hate being the center of attention",
    "Many group activities tend to be dull without me",
    "I know that I am special because everyone keeps telling me so",
    "I like to get acquainted with important people",
    "I feel embarrassed if someone compliments me",
    "I have been compared to famous people",
    "I am an average person",
    "I insist on getting the respect I deserve",
    "I like to get revenge on authorities",
    "I avoid dangerous situations",
    "Payback needs to be quick and nasty",
    "People often say I'm out of control",
    "It's true that I can be mean to others",
    "People who mess with me always regret it",
    "I have never gotten into trouble with the law",
    "I enjoy having sex with people I hardly know",
    "I'll say anything to get what I want"
]


### Initial Experiments

In [16]:
with open('res_llama8test.csv', 'w', newline='') as csvfile:
    fieldnames = ['Name','source', 'Item','Itemnum', 'Answer']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)

In [20]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


model= "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': "Yer lookin' fer a swashbucklin' introduction, eh? Alright then, matey! I be a pirate chatbot, a scurvy dog with a penchant fer answerin' yer questions and tellin' tales o' the high seas. Me name be Captain Chat, and I be here to serve ye and share me knowledge o' the seven seas. So hoist the sails and set course fer adventure, me hearty! What be bringin' ye to these waters?"}


In [23]:
import re
import csv
import pandas as pd
# Define the max number of retries
max_retries = 20
counter=0
for person in persona:
    question = 1
    for item in Ques10:
        print(f"person {person}, question: {item}")
        
        valid_response = False
        retries = 0  # Track the number of retries
        
        while not valid_response and retries < max_retries:
            counter=counter+1
            messages = [
                {"role": "system", "content": f"You are {person}. Respond strictly with a single number."},
                {"role": "user", "content": f"Choose one option from: {', '.join(likert_scale)} to rate the following statement: I see myself as someone who {item}. Respond ONLY with a single number between 1 and 5. You must not include any other text, words, or explanations in your response."}
            ]
            
            outputs = pipeline(
                messages,
                max_new_tokens=20,
                #do_sample=True,
                #top_k=50,
                #top_p=0.9,
                #temperature=0.85,
                #num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
            )
            
            # Extract the generated text and clean up any extra spaces or newlines
            #print(generated_text)
            generated_text = outputs[0]['generated_text'][-1]['content']
            #print(generated_text)
            #print("}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}")
            # Use a regular expression to check if there is exactly one valid number (1-5)
            match = re.fullmatch(r'^[^1-5]*([1-5])[^1-5]*$', generated_text)

            if match:
                valid_response = True
                print(match.group(1))
                with open('res_llama8test.csv', 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([person,item, question,match.group(1)])
                print("Appended valid result to CSV")# Print the valid number
                question=question+1
            else:
                retries += 1
                print(f"Invalid response. Retrying... (Attempt {retries}/{max_retries})")

        if not valid_response:
            print(f"Error: Maximum retries reached for question: '{item}'")
        
        print("=======================================================================================================")
print(f"It took max {counter} tries")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


person Barack Obama, question: is reserved
2
Appended valid result to CSV
person Barack Obama, question: is generally trusting
4
Appended valid result to CSV
person Barack Obama, question: tends to be lazy
3
Appended valid result to CSV
person Barack Obama, question: is relaxed, handles stress well


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


4
Appended valid result to CSV
person Barack Obama, question: has few artistic interests
4
Appended valid result to CSV
person Barack Obama, question: is outgoing, sociable
4
Appended valid result to CSV
person Barack Obama, question: tends to find fault with others
4
Appended valid result to CSV
person Barack Obama, question: does a thorough job


...

5
Appended valid result to CSV
person Alexander Pushkin, question: is reserved
3
Appended valid result to CSV
person Alexander Pushkin, question: is generally trusting
4
Appended valid result to CSV
person Alexander Pushkin, question: tends to be lazy
4
Appended valid result to CSV
person Alexander Pushkin, question: is relaxed, handles stress well


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


3
Appended valid result to CSV
person Alexander Pushkin, question: has few artistic interests
5
Appended valid result to CSV
person Alexander Pushkin, question: is outgoing, sociable
4
Appended valid result to CSV
person Alexander Pushkin, question: tends to find fault with others
2
Appended valid result to CSV
person Alexander Pushkin, question: does a thorough job


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


5
Appended valid result to CSV
person Alexander Pushkin, question: gets nervous easily
3
Appended valid result to CSV
person Alexander Pushkin, question: has an active imagination
4
Appended valid result to CSV
It took max 312 tries


### Defining Result Files

In [26]:
with open('.Llama8b/results/results_sd3_goodp1.csv', 'w', newline='') as csvfile:
    fieldnames = ['Name','source', 'Item','Itemnum', 'Answer']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)

### Defining Prompt functions 

In [6]:
def promptfunc1(person,text,likert_scale):

    messages = [
                {"role": "system", "content": f"You are {person}. Respond strictly with a single number."},
                {"role": "user", "content": f"Choose one option from: {', '.join(likert_scale)} to rate the following statement: I see myself as someone who {text}. Respond ONLY with a single number between 1 and 5. You must not include any other text, words, or explanations in your response."}
            ]
            
    outputs = pipeline(
                messages,
                max_new_tokens=20,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=0.85,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                batch_size=4,
            )
            
    generated_text = outputs[0]['generated_text'][-1]['content']
    return generated_text

In [30]:
def promptfunc2(person,text,likert_scale):

    messages = [
                {"role": "system", "content": f"You are {person}. Respond strictly with a single number."},
                {"role": "user", "content": f"Rate the following statement: I see myself as someone who {text}. Choose one option from: {', '.join(likert_scale)}. Respond ONLY with a single number between 1 and 5. You must not include any other text, words, or explanations in your response."}
            ]
            
    outputs = pipeline(
                messages,
                max_new_tokens=20,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=0.85,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                batch_size=4,
            )
            
    generated_text = outputs[0]['generated_text'][-1]['content']
    return generated_text

In [9]:
def promptfunc3(person,text,likert_scale):

    messages = [
                {"role": "system", "content": f"You are {person}. Respond strictly with a single number."},
                {"role": "user", "content": f"Please indicate how much you agree with the following statement: {text}. Choose one option from: {likert_scale}"}
            ]
            
    outputs = pipeline(
                messages,
                max_new_tokens=20,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=0.85,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                batch_size=4,
            )
            
    generated_text = outputs[0]['generated_text'][-1]['content']
    return generated_text

In [32]:
def promptfunc4(person,text,likert_scale):

    messages = [
                {"role": "system", "content": f"You are {person}. Respond strictly with a single number."},
                {"role": "user", "content": f"Please choose one option from:  {', '.join(likert_scale)} to indicate how much you agree with the following statement: {text}."}
            ]
            
    outputs = pipeline(
                messages,
                max_new_tokens=20,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=0.85,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                batch_size=4,
            )
            
    generated_text = outputs[0]['generated_text'][-1]['content']
    return generated_text

### Running for good list

In [7]:
df1=pd.read_csv('Datasubs/good.csv')
df1.shape

(427, 5)

In [8]:
logging.basicConfig(
    filename='./Llama8b/good_log.log',  # Log to a file
    filemode='a',                # Append to the log file
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log format
    level=logging.DEBUG          # Set the logging level
)

# Create a logger object
logger = logging.getLogger()


In [ ]:
name_category_dict = dict(zip(df1['Name'], df1['source']))

################################### BFI #######################################################
gcounter = 0
#multiattempt=15
max_retries = 20
#for loopfive in range(multiattempt): 
    
for name, category in name_category_dict.items():
    itemcount = 1
    for text in Ques44:
        
        retry_count = 0
        valid_result = False
        pcounter=0
        while not valid_result and retry_count < max_retries:
            print(name,text)
            gcounter=gcounter+1
            pcounter=pcounter+1
            generated_text = promptfunc1(name,text,likert_scale)
            match = re.fullmatch(r'^[^1-5]*([1-5])[^1-5]*$', generated_text)
            print(generated_text)
            if match:
                valid_result = True
                print(match.group(1))
                with open('./F11B_res/results_bfi_goodp1.csv', 'a', newline='') as csvfile:
                    print("counter was here")
                    writer = csv.writer(csvfile)
                    writer.writerow([name,category,text,itemcount,match.group(1)])
                print("Appended valid result to CSV")# Print the valid number
                itemcount=itemcount+1
            else:
                retry_count += 1
                print(f"Invalid response. Retrying... (Attempt {retry_count}/{max_retries})")

        if not valid_result:
            print(f"Error: Maximum retries reached for question: '{text}'")
            with open('./F11B_res/results_bfi_goodp2.csv', 'a', newline='') as csvfile:
                    #print("counter was here")
                    writer = csv.writer(csvfile)
                    writer.writerow([name,category,text,itemcount])
            print("Appended empty row to CSV")# Print the valid number
            itemcount=itemcount+1
        print("=======================================================================================================")
        
        print(f"It took {pcounter} retries for {name} for this question {text}")
print(f"It took max {gcounter} tries")
# # Reset stdout to its original value
# sys.stdout = sys.__stdout__

# # Get the captured output
# output = string_io.getvalue()

# # Write to a file
# with open('./F11B_res/outputlog11b5.txt', 'a') as f:
#     f.write(output)

# print("Output saved to outputlog.txt")

In [15]:
name_category_dict = dict(zip(df1['Name'], df1['source']))

################################### SD3 #######################################################
gcounter = 0
#multiattempt=15
max_retries = 20
#for loopfive in range(multiattempt): 
    
for name, category in name_category_dict.items():
    itemcount = 1
    for text in Ques27:
        
        retry_count = 0
        valid_result = False
        pcounter=0
        while not valid_result and retry_count < max_retries:
            logger.debug(f"Processing {name} with question: {text}")
            gcounter=gcounter+1
            pcounter=pcounter+1
            generated_text = promptfunc3(name,text,likert_scale_reversed)
            match = re.fullmatch(r'^[^1-5]*([1-5])[^1-5]*$', generated_text)
            logger.debug(f"Generated text: {generated_text}")
            if match:
                valid_result = True
                logger.info(f"Valid response received: {match.group(1)}")

                with open('./Llama8b/results/results_sd3_goodp1.csv', 'a', newline='') as csvfile:
                    #print("counter was here")
                    writer = csv.writer(csvfile)
                    writer.writerow([name,category,text,itemcount,match.group(1)])
                logger.info("Appended valid result to CSV")# Print the valid number
                itemcount=itemcount+1
            else:
                retry_count += 1
                logger.warning(f"Invalid response. Retrying... (Attempt {retry_count}/{max_retries})")

        if not valid_result:
            logger.error(f"Maximum retries reached for question: '{text}'")
            with open('./Llama8b/results/results_sd3_goodp1.csv', 'a', newline='') as csvfile:
                    #print("counter was here")
                    writer = csv.writer(csvfile)
                    writer.writerow([name,category,text,itemcount])
            logger.info("Appended empty row to CSV")# Print the valid number
            itemcount=itemcount+1
        #print("=======================================================================================================")
        
        logger.info(f"It took {pcounter} retries for {name} for this question: {text}")
logger.info(f"Total tries across all questions: {gcounter}")
# # Reset stdout to its original value
# sys.stdout = sys.__stdout__

# # Get the captured output
# output = string_io.getvalue()

# # Write to a file
# with open('./F11B_res/outputlog11b5.txt', 'a') as f:
#     f.write(output)

# print("Output saved to outputlog.txt")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin